In [16]:
%%sh
pip install -U -q transformers torch torchvision matplotlib

In [ ]:
import torch
from torchvision.datasets import MNIST
from torchvision.transforms import v2 as transforms
from matplotlib import pyplot as plt

transform = transforms.Compose([
    transforms.ToImage(),
    transforms.ToDtype(torch.float32, scale=True),
    transforms.Normalize((0.1307,), (0.3081,))
])
batch_size = 5
kwargs = {'batch_size': batch_size}

dataset1 = MNIST("./data", train=True, download=True, transform=transform)
dataset2 = MNIST("./data", train=False, transform=transform)
train_loader = torch.utils.data.DataLoader(dataset1, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(dataset2, **kwargs)

plt.imshow(dataset1[0][0].squeeze())
plt.axis("off")
plt.show()

In [ ]:
import torch

from simple_model.pytorch_model import Model

device = "cuda" if torch.cuda.is_available() else "cpu"
net = Model()
net = net.to(device)
net

In [ ]:
import torch.nn.functional as F
import torch.optim as optim
from torch import zeros
from torch.optim.lr_scheduler import StepLR
from tqdm import tqdm

EPOCH = 1

optimizer = optim.Adadelta(net.parameters(), lr=1.0)
scheduler = StepLR(optimizer, step_size=1, gamma=0.7)
net.train()

for epoch in range(EPOCH):
    print(f"EPOC {epoch + 1}/{EPOCH}")
    for data, target in tqdm(train_loader):
        data, target_tensor = data.to(device), zeros(target.shape[0], 10).to(device)
        for batch_id, idx in enumerate(target):
            target_tensor[batch_id, idx] = 1.0
        optimizer.zero_grad()
        output = net(data)
        loss = F.cross_entropy(output, target_tensor)
        loss.backward()
        optimizer.step()

In [ ]:
import torch
from tqdm import tqdm

net.eval()
test_loss = 0
correct = 0
with torch.no_grad():
    for data, target in tqdm(test_loader):
        target = target.to(device)
        data, target_tensor = data.to(device), zeros(target.shape[0], 10).to(device)
        for batch_id, idx in enumerate(target):
            target_tensor[batch_id, idx] = 1.0
        output = net(data)
        test_loss += F.cross_entropy(output, target_tensor, reduction='sum').item()  # sum up batch loss
        pred = net.get_guess(output)  # get the index of the max log-probability
        correct += pred.eq(target.view_as(pred)).sum().item()

test_loss /= len(test_loader.dataset)

print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(test_loader.dataset),
    100. * correct / len(test_loader.dataset)))

torch.save(net.state_dict(), "simple_model.pth")

In [ ]:
from IPython.display import display
from torchvision.datasets import MNIST

data = MNIST('./data')
test = [data[0][0], data[1][0]]

display(test[0])
display(test[1])

In [7]:
from simple_model import SimpleModel, SimpleModelConfig

SimpleModelConfig.register_for_auto_class("AutoConfig")
SimpleModel.register_for_auto_class("AutoModel")
SimpleModel.register_for_auto_class("AutoModelForImageClassification")

simple_model_config = SimpleModelConfig()
simple_model = SimpleModel(simple_model_config)
simple_model.model.load_state_dict(net.state_dict())

simple_model.save_pretrained("simple-model")

In [ ]:
from simple_model import SimpleModelProcessor

SimpleModelProcessor.register_for_auto_class("AutoProcessor")
SimpleModelProcessor.register_for_auto_class("AutoImageProcessor")

processor = SimpleModelProcessor()
processor.save_pretrained("simple-model")

In [ ]:
with torch.no_grad():
    test_input = processor(test)
    test_output = simple_model(**test_input)

test_output

In [ ]:
from transformers import pipeline

pipe = pipeline(
    "image-classification",
    model=simple_model,
    image_processor=processor,
    trust_remote_code=True,
)
pipe.save_pretrained("simple-model")
pipe(test)

In [ ]:
from transformers import AutoConfig, AutoModel, AutoProcessor

output_processor = AutoProcessor.from_pretrained(
    "simple-model",
    trust_remote_code=True,
)
output_model = AutoModel.from_pretrained(
    "simple-model",
    # Loading config manually as it is registered in library
    config=AutoConfig.from_pretrained("simple-model"),
    trust_remote_code=True
)

with torch.no_grad():
    test_input = processor(test)
    test_output = output_model(**test_input)

test_output

In [ ]:
from transformers import pipeline

output_pipeline = pipeline(
    "image-classification",
    model="simple-model",
    image_processor="simple-model",
    trust_remote_code=True,
)
output_pipeline(test)

In [13]:
%%sh
pip install -U -q "optimum[exporters]"

In [ ]:
from importlib import reload
import simple_model

reload(simple_model)
reload(simple_model.configuration_simple_model)
reload(simple_model.modeling_simple_model)
reload(simple_model.processing_simple_model)
reload(simple_model.pytorch_model)

In [ ]:
from pathlib import Path

from optimum.exporters import TasksManager
from optimum.exporters.onnx import export
from transformers import AutoConfig, AutoModel, AutoProcessor

from simple_model.configuration_simple_model import SimpleModelOnnxConfig

output_config = AutoConfig.from_pretrained("simple-model")
output_model = AutoModel.from_pretrained(
    "simple-model",
    config=output_config,
    trust_remote_code=True
)
onnx_config = SimpleModelOnnxConfig(output_config, task="image-classification")
print(f"ONNX Inputs: {onnx_config.inputs}")
print(f"ONNX Outputs: {onnx_config.outputs}")
print(f"ONNX Opset: {onnx_config.DEFAULT_ONNX_OPSET}")

TasksManager._SUPPORTED_MODEL_TYPE["simple-model"] = {
    "onnx": {
        "image-classification": SimpleModelOnnxConfig
    }
}

onnx_config_constructor = TasksManager.get_exporter_config_constructor(
    "onnx",
    output_model,
    task="image-classification"
)
onnx_config = onnx_config_constructor(output_model.config, task="image-classification")

onnx_inputs, onnx_outputs = export(
    output_model,
    onnx_config,
    Path("simple-model.onnx"),
    onnx_config.DEFAULT_ONNX_OPSET,
    input_shapes={
        "num_channels": 1,
        "height": 28,
        "width": 28
    },
)